In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sst
import os
import scipy.stats as sst
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import silhouette_score

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp') # LMS Zone data

In [ ]:
## Modal split travel behaviour
ovin_tb = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_ovin_travel_behaviour.csv', index_col=0)
lms_tb = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_lms_modal_split.csv', index_col=0)

In [ ]:
ovin = pd.read_csv(parent_dir + '\\Data\\New\\Ovin_final.csv', index_col=0)

In [ ]:
# Density
dens = pd.read_csv((parent_dir + '\\Data\\New\\lms_zone_density.csv'), index_col=0)

# Diversity
landuse = pd.read_csv((parent_dir + '\\Data\\New\\lms_diversity_lu.csv'), index_col=0)
hist = pd.read_csv((parent_dir + '\\Data\\New\\lms_zone_historical.csv'), index_col=0)

# Design
design = pd.read_csv((parent_dir + '\\Data\\New\\lms_zone_design.csv'), index_col=0) 

# Destination accessibility
dest = pd.read_csv((parent_dir + '\\Data\\New\\lms_zone_dest_access.csv'), index_col=0) 

# Distance to transit
transit = pd.read_csv((parent_dir + '\\Data\\New\\lms_zone_transit.csv'), index_col=0) 

# Demography
demo = pd.read_csv((parent_dir + '\\Data\\New\\zone_demographics.csv'), index_col=0)  

## Select small selection of zones to test

In [ ]:
# ids = [627, 934, 777, 452, 680, 745, 822, 21, 1404, 187, 313, 1363, 170, 40, 1391, 172]
ids = [626, 933, 776, 451, 679, 744, 821, 20, 1403, 186, 312, 1362, 169, 39, 1390, 171]

## Uncomment if testing all the zones
# ids = np.arange(1406) 

## Clustering

To start, do simple hierarchical clustering based on a few attributes. It doesn't have to be realistic.

In [ ]:
def scale_data(features, method='scale'):
    """
    Scale the data to 0-1 or normalize the data

    Parameters:
    features: numpy array containing all the features
    method: the type of data-scaling, string

    Returns:
    The transformed data as a numpy array
    """
    if method == 'scale':
        scaler = MinMaxScaler()
    elif method == 'standard':
        scaler = StandardScaler()
    else:
        return 'Not a valid scaler'
    
    return scaler.fit_transform(features)

In [ ]:
x1 = dest.loc[ids, 'Dist_to_center']
x2 = design.loc[ids, 'Road_density']
x3 = landuse.loc[ids, 'Entropy']
x4 = transit.loc[ids, 'Distance_station']

In [ ]:
data = list(zip(x1, x2, x3, x4))

In [ ]:
data = scale_data(data)

In [ ]:
linkage_data = linkage(data, method='ward', metric='euclidean')

In [ ]:
dendrogram(linkage_data, color_threshold=5);
plt.title('Dendrogram test cluster')
plt.xlabel('Cluster number')
plt.xticks([], [])

In [ ]:
sil_list = []

for k in range(2, 16):
    hier_cluster = AgglomerativeClustering(n_clusters=k, metric='euclidean', linkage='ward')
    
    labels = hier_cluster.fit_predict(data)

    sil = silhouette_score(data, labels, metric = 'euclidean') 
    sil_list.append(sil)

In [ ]:
plt.plot(np.arange(2, 16), sil_list)
plt.ylabel('Silhouette score')
plt.xlabel('Number of clusters')
plt.title('Silhouette score of clusters')
plt.xticks(np.arange(2, 16))
plt.grid();

Silhouette score shows that 3 clusters would be the best. 

In [ ]:
hier_cluster = AgglomerativeClustering(n_clusters=3, metric='euclidean', linkage='ward')
labels = hier_cluster.fit_predict(data)

In [ ]:
x1 = data[:, 0]
x2 = data[:, 1]
x3 = data[:, 2]
x4 = data[:, 3]


In [ ]:
f, ax = plt.subplots(2, 3)
f.set_figwidth(12)
f.set_figheight(7)
ax[0, 0].scatter(x1, x2, c=labels)
ax[0, 0].set_xlabel('Distance to center')
ax[0, 0].set_ylabel('Road density')

ax[0, 1].scatter(x1, x3, c=labels)
ax[0, 1].set_xlabel('Distance to center')
ax[0, 1].set_ylabel('Entropy')


ax[0, 2].scatter(x1, x4, c=labels)
ax[0, 2].set_xlabel('Distance to center')
ax[0, 2].set_ylabel('Distance to station')

ax[1, 0].scatter(x2, x3, c=labels)
ax[1, 0].set_xlabel('Road density')
ax[1, 0].set_ylabel('Entropy')

ax[1, 1].scatter(x2, x4, c=labels)
ax[1, 1].set_xlabel('Road density')
ax[1, 1].set_ylabel('Distance to station')


scatter = ax[1, 2].scatter(x3, x4, c=labels)
ax[1, 2].set_xlabel('Entropy')
ax[1, 2].set_ylabel('Distance to station')

legend1 = ax[1, 2].legend(*scatter.legend_elements(),
                    loc="upper right", title="Clusters")
ax[1, 2].add_artist(legend1)

# f.suptitle('Clustering based on 4 variables');

# plt.legend()

In [ ]:
df = pd.DataFrame(data)
df['Cluster'] = labels

In [ ]:
df.index = ids

In [ ]:
df

## Get OViN data points from respective zone clusters

Get indices for each cluster

In [ ]:
id0 = df[df.Cluster == 0].index
id1 = df[df.Cluster == 1].index
id2 = df[df.Cluster == 2].index


Select OViN trips based on departure zones. Don't take into account the double people or the weightfactors

In [ ]:
o0 = ovin[ovin.AankZone.isin(id0 + 1)]
o1 = ovin[ovin.AankZone.isin(id1 + 1)]
o2 = ovin[ovin.AankZone.isin(id2 + 1)]


And get relevant categories

In [ ]:
o0 = o0[['HHPers', 'Leeftijd', 'HHBestInk', 'HHAuto', 'KHvm']].dropna()
o1 = o1[['HHPers', 'Leeftijd', 'HHBestInk', 'HHAuto', 'KHvm']].dropna()
o2 = o2[['HHPers', 'Leeftijd', 'HHBestInk', 'HHAuto', 'KHvm']].dropna()


In [ ]:
o0['Cluster'] = 0
o1['Cluster'] = 1
o2['Cluster'] = 2


In [ ]:
o0.loc[o0.KHvm > 2, 'KHvm'] = 0
o0.loc[o0.KHvm > 0, 'KHvm'] = 1

o1.loc[o1.KHvm > 2, 'KHvm'] = 0
o1.loc[o1.KHvm > 0, 'KHvm'] = 1

o2.loc[o2.KHvm > 2, 'KHvm'] = 0
o2.loc[o2.KHvm > 0, 'KHvm'] = 1

In [ ]:
o0.mean(), o1.mean(), o2.mean()

Alternatively, use zones itself

In [ ]:
# o0 = demo[['Household_Size', 'Age_average', 'Income_hh_average', 'Cars_HH']].iloc[id0]
# o1 = demo[['Household_Size', 'Age_average', 'Income_hh_average', 'Cars_HH']].iloc[id1]
# o2 = demo[['Household_Size', 'Age_average', 'Income_hh_average', 'Cars_HH']].iloc[id2]

In [ ]:
# o0['Cluster'] = 0
# o1['Cluster'] = 1
# o2['Cluster'] = 2

In [ ]:
# o0['Car_ovin'] = ovin_tb.iloc[id0]['Car_passenger_o'] + ovin_tb.iloc[id0]['Car_driver_o']
# o1['Car_ovin'] = ovin_tb.iloc[id1]['Car_passenger_o'] + ovin_tb.iloc[id1]['Car_driver_o']
# o2['Car_ovin'] = ovin_tb.iloc[id2]['Car_passenger_o'] + ovin_tb.iloc[id2]['Car_driver_o']


In [ ]:
# o0 = o0.dropna()
# o1 = o1.dropna()
# o2 = o2.dropna()

In [ ]:
# o0.mean(), o1.mean(), o2.mean()

## Prospensity score matching

In [ ]:
_, p01 = sst.ttest_ind(o0.KHvm, o1.KHvm)
_, p02 = sst.ttest_ind(o0.KHvm, o2.KHvm)
_, p12 = sst.ttest_ind(o1.KHvm, o2.KHvm)

# _, p01 = sst.ttest_ind(o0.Car_ovin, o1.Car_ovin)
# _, p02 = sst.ttest_ind(o0.Car_ovin, o2.Car_ovin)
# _, p12 = sst.ttest_ind(o1.Car_ovin, o2.Car_ovin)


In [ ]:
p01, p02, p12

All p-values are lower than 0.05 so the differences seem to be significant for now.

Next step is to calculate the prospensity scores

In [ ]:
df01 = pd.concat([o0, o1])
df02 = pd.concat([o0, o2])
df12 = pd.concat([o1, o2])


In [ ]:
df01 = df01.reset_index()
df02 = df02.reset_index()
df12 = df12.reset_index()

In [ ]:
df02['Cluster'] = df02['Cluster'] / 2
df12['Cluster'] = df12['Cluster'] - 1


In [ ]:
X01 = df01.iloc[:, 1:-2]
X02 = df02.iloc[:, 1:-2]
X12 = df12.iloc[:, 1:-2]

y01 = df01.iloc[:, -1]
y02 = df02.iloc[:, -1]
y12 = df12.iloc[:, -1]

# y01 = df01.iloc[:, -2]
# y02 = df02.iloc[:, -2]
# y12 = df12.iloc[:, -2]


In [ ]:
lr01 = LogisticRegression()
lr02 = LogisticRegression()
lr12 = LogisticRegression()

lr01.fit(X01, y01)
lr02.fit(X02, y02)
lr12.fit(X12, y12)


In [ ]:
coeffs = pd.DataFrame({
    'column':X01.columns.to_numpy(),
    'coeff01':lr01.coef_.ravel(),
    'coeff02':lr02.coef_.ravel(),
    'coeff12':lr12.coef_.ravel(),

})
coeffs

Now make predictions (i.e. the prospensity score)

In [ ]:
prob01 = lr01.predict_proba(X01)
prob02 = lr02.predict_proba(X02)
prob12 = lr12.predict_proba(X12)


In [ ]:
df01['ps'] = prob01[:, 1]
df02['ps'] = prob02[:, 1]
df12['ps'] = prob12[:, 1]


Now check if there is overlap between the groups. Otherwise you cannot match

In [ ]:
f, ax = plt.subplots(1, 3)
f.set_figwidth(12)
f.set_figheight(3)

ax[0].hist(df01[df01['Cluster'] == 0]['ps'], bins=20, alpha=0.8, label='cluster 0')
ax[0].hist(df01[df01['Cluster'] == 1]['ps'], bins=20, alpha=0.5, label='cluster 1')
ax[0].set_title('Propensity score cluster 0 an 1')
ax[0].set_ylabel('Number of observations')
ax[0].set_xlabel('Propensity score')
ax[0].legend()

ax[1].hist(df02[df02['Cluster'] == 0]['ps'], bins=20, alpha=0.8, label='cluster 0')
ax[1].hist(df02[df02['Cluster'] == 1]['ps'], bins=20, alpha=0.5, label='cluster 2')
ax[1].set_title('Propensity score cluster 0 an 2')
# ax[1].set_ylabel('Number of observations')
ax[1].set_xlabel('Propensity score')
ax[1].legend()

ax[2].hist(df12[df12['Cluster'] == 0]['ps'], bins=20, alpha=0.8, label='cluster 1')
ax[2].hist(df12[df12['Cluster'] == 1]['ps'], bins=20, alpha=0.5, label='cluster 2')
ax[2].set_title('Propensity score cluster 1 an 2')
# ax[2].set_ylabel('Number of observations')
ax[2].set_xlabel('Propensity score')
ax[2].legend();

There seems to be sufficient overlap

In [ ]:
len(o0), len(o1), len(o2)

In [ ]:
n_neigh = 25
caliper = 0.01

knn01 = NearestNeighbors(n_neighbors=n_neigh, radius=caliper)
knn01.fit(df01[['ps']])

knn02 = NearestNeighbors(n_neighbors=n_neigh, radius=caliper)
knn02.fit(df02[['ps']])

knn12 = NearestNeighbors(n_neighbors=n_neigh, radius=caliper)
knn12.fit(df12[['ps']])

In [ ]:
d01, n01 = knn01.kneighbors(df01[['ps']])
d02, n02 = knn02.kneighbors(df02[['ps']])
d12, n12 = knn12.kneighbors(df12[['ps']])


In [ ]:
match01 = []
match02 = []
match12 = []

df = [df01, df02, df12]
neighbour = [n01, n02, n12]
match = [match01, match02, match12]

for i in range(3):
    for current_i, row in df[i].iterrows():
        if row.Cluster == 0:
            df[i].loc[current_i, 'matched'] = np.nan
        else:
            for idx in neighbour[i][current_i, :]:
                if (current_i != idx) and (df[i].iloc[idx].Cluster == 0):
                    if idx not in match[i]:
                        df[i].loc[current_i, 'matched'] = idx
                        match[i].append(idx)
                        break

In [ ]:
print(len(match01), len(df01[df01.Cluster == 0]), len(df01[df01.Cluster == 1]))
print(len(match02), len(df02[df02.Cluster == 0]), len(df02[df02.Cluster == 1]))
print(len(match12), len(df12[df12.Cluster == 0]), len(df12[df12.Cluster == 1]))


In [ ]:
df01_m = df01.dropna(subset=['matched'])
df02_m = df02.dropna(subset=['matched'])
df12_m = df12.dropna(subset=['matched'])

In [ ]:
control_id = df01_m.matched
control_id = control_id.astype(int)
df01_c = df01.loc[control_id, :]

df01_f = pd.concat([df01_m, df01_c])


control_id = df02_m.matched
control_id = control_id.astype(int)
df02_c = df02.loc[control_id, :]

df02_f = pd.concat([df02_m, df02_c])

control_id = df12_m.matched
control_id = control_id.astype(int)
df12_c = df12.loc[control_id, :]

df12_f = pd.concat([df12_m, df12_c])

In [ ]:
df12_f

Now check p-value and means

In [ ]:
df02_c.mean(), df02_m.mean()

# , o1.mean(), o2.mean()

In [ ]:
_, p01_n = sst.ttest_ind(df01_m.KHvm, df01_c.KHvm)
_, p02_n = sst.ttest_ind(df02_m.KHvm, df02_c.KHvm)
_, p12_n = sst.ttest_ind(df12_m.KHvm, df12_c.KHvm)

# _, p01_n = sst.ttest_ind(df01_m.Car_ovin, df01_c.Car_ovin)
# _, p02_n = sst.ttest_ind(df02_m.Car_ovin, df02_c.Car_ovin)
# _, p12_n = sst.ttest_ind(df12_m.Car_ovin, df12_c.Car_ovin)

In [ ]:
p01, p02, p12

In [ ]:
p01_n, p02_n, p12_n

Now only the clusters 0 and 1 and 1 and 2 are significant

Check from which areas this was

In [ ]:
id0

In [ ]:
zones.iloc[id0].GEM_NAAM

In [ ]:
zones.iloc[id1].GEM_NAAM

In [ ]:
zones.iloc[id2].GEM_NAAM

## Travel behaviour comparisons

In [ ]:
lms_tb2 = lms_tb.iloc[:, 1:8].copy()
lms_tb2.iloc[:, 3] = lms_tb2.iloc[:, 3:5].sum(axis=1)
lms_tb2 = lms_tb2.drop(columns='Tram/Metro_o')

In [ ]:
pop = demo.loc[id0, 'Tot_population']
r0 = ovin_tb.iloc[id0, 1:7].multiply(pop, axis='index').sum() / pop.sum()
r0

In [ ]:
pop = demo.loc[id1, 'Tot_population']
r1 = ovin_tb.iloc[id1, 1:7].multiply(pop, axis='index').sum() / pop.sum()
r1

In [ ]:
pop = demo.loc[id2, 'Tot_population']
r2 = ovin_tb.iloc[id2, 1:7].multiply(pop, axis='index').sum() / pop.sum()
r2

Now for LMS

In [ ]:
pop = demo.loc[id0, 'Tot_population']
l0 = lms_tb2.iloc[id0].multiply(pop, axis='index').sum() / pop.sum()
l0

In [ ]:
pop = demo.loc[id1, 'Tot_population']
l1 = lms_tb2.iloc[id1].multiply(pop, axis='index').sum() / pop.sum()
l1

In [ ]:
pop = demo.loc[id2, 'Tot_population']
l2 = lms_tb2.iloc[id2].multiply(pop, axis='index').sum() / pop.sum()
l2

Also calculate for corrected OViN

In [ ]:
counts = ovin.iloc[df01_m['index']].KHvm.value_counts()
counts = counts.loc[[1, 2, 3, 4, 6, 7]]
counts1_01 = counts / counts.sum() * 100
counts1_01

# counts1_01 = ovin_tb.iloc[df01_m['index']].iloc[:, 1:7].mean()
# l1_01 = lms_tb2.iloc[df01_m['index']].mean()

In [ ]:
counts = ovin.iloc[df01_c['index']].KHvm.value_counts()
counts = counts.loc[[1, 2, 3, 4, 6, 7]]
counts0_01 = counts / counts.sum() * 100
counts0_01

# counts0_01 = ovin_tb.iloc[df01_c['index']].iloc[:, 1:7].mean()
# l0_01 = lms_tb2.iloc[df01_c['index']].mean()

In [ ]:
counts = ovin.iloc[df02_m['index']].KHvm.value_counts()
counts = counts.loc[[1, 2, 3, 4, 6, 7]]
counts2_02 = counts / counts.sum() * 100
counts2_02

# counts2_02 = ovin_tb.iloc[df02_m['index']].iloc[:, 1:7].mean()
# l2_02 = lms_tb2.iloc[df02_m['index']].mean()

In [ ]:
counts = ovin.iloc[df02_c['index']].KHvm.value_counts()
counts = counts.loc[[1, 2, 3, 4, 6, 7]]
counts0_02 = counts / counts.sum() * 100
counts0_02

# counts0_02 = ovin_tb.iloc[df02_c['index']].iloc[:, 1:7].mean()
# l0_02 = lms_tb2.iloc[df02_c['index']].mean()

In [ ]:
counts = ovin.iloc[df12_m['index']].KHvm.value_counts()
counts = counts.loc[[1, 2, 3, 4, 6, 7]]
counts2_12 = counts / counts.sum() * 100
counts2_12

# counts2_12 = ovin_tb.iloc[df12_m['index']].iloc[:, 1:7].mean()
# l2_12 = lms_tb2.iloc[df12_m['index']].mean()

In [ ]:
counts = ovin.iloc[df12_c['index']].KHvm.value_counts()
counts = counts.loc[[1, 2, 3, 4, 6, 7]]
counts1_12 = counts / counts.sum() * 100
counts1_12

# counts1_12 = ovin_tb.iloc[df12_c['index']].iloc[:, 1:7].mean()
# l1_12 = lms_tb2.iloc[df12_c['index']].mean()

In [ ]:
f, ax = plt.subplots(1, 3)
f.set_figwidth(12)

x = np.arange(6)

ax[0].bar(x - 0.15, counts0_01, width=0.25, label='Cluster 0', color='firebrick')
# ax[0].bar(x, r2, width=0.2, label='Cluster 2')
ax[0].bar(x + 0.15, counts1_01, width=0.25, label='Cluster 1', color='royalblue')

ax[0].set_xticks(x, labels=['Car driver', 'Car passenger', 'Train', 
                      'BTM', 'Cycling', 'Walking'], rotation=45,
                      ha='right')

f.suptitle('Modal split OViN, corrected by demography')
ax[0].set_ylabel('Percentage of mode use')
ax[0].legend()
ax[0].set_yticks(np.arange(0, 50, 5))
ax[0].grid(axis='y')
ax[0].set_axisbelow(True)


ax[1].bar(x - 0.15, counts0_02, width=0.25, label='Cluster 0', color='firebrick')
# ax[0].bar(x, r2, width=0.2, label='Cluster 2')
ax[1].bar(x + 0.15, counts2_02, width=0.25, label='Cluster 2', color='darkgreen')

ax[1].set_xticks(x, labels=['Car driver', 'Car passenger', 'Train', 
                      'BTM', 'Cycling', 'Walking'], rotation=45,
                      ha='right')

# ax[1].set_title('Modal split OViN, corrected by demography')
ax[1].set_ylabel('Percentage of mode use')
ax[1].legend()
ax[1].set_yticks(np.arange(0, 50, 5))
ax[1].grid(axis='y')
ax[1].set_axisbelow(True)

ax[2].bar(x - 0.15, counts2_12, width=0.25, label='Cluster 2', color='darkgreen')
# ax[0].bar(x, r2, width=0.2, label='Cluster 2')
ax[2].bar(x + 0.15, counts1_12, width=0.25, label='Cluster 1', color='royalblue')

ax[2].set_xticks(x, labels=['Car driver', 'Car passenger', 'Train', 
                      'BTM', 'Cycling', 'Walking'], rotation=45,
                      ha='right')

# ax[2].set_title('Modal split OViN, corrected by demography')
ax[2].set_ylabel('Percentage of mode use')
ax[2].legend()
ax[2].set_yticks(np.arange(0, 50, 5))
ax[2].grid(axis='y')
ax[2].set_axisbelow(True)


# # LMS
# f, ax = plt.subplots(1, 3)
# f.set_figwidth(12)

# x = np.arange(6)

# ax[0].bar(x - 0.15, l0_01, width=0.25, label='Cluster 0', color='firebrick')
# # ax[0].bar(x, r2, width=0.2, label='Cluster 2')
# ax[0].bar(x + 0.15, l1_01, width=0.25, label='Cluster 1', color='royalblue')

# ax[0].set_xticks(x, labels=['Car driver', 'Car passenger', 'Train', 
#                       'BTM', 'Cycling', 'Walking'], rotation=45,
#                       ha='right')

# f.suptitle('Modal split LMS, corrected by demography')
# ax[0].set_ylabel('Percentage of mode use')
# ax[0].legend()
# ax[0].set_yticks(np.arange(0, 50, 5))
# ax[0].grid(axis='y')
# ax[0].set_axisbelow(True)


# ax[1].bar(x - 0.15, l0_02, width=0.25, label='Cluster 0', color='firebrick')
# # ax[0].bar(x, r2, width=0.2, label='Cluster 2')
# ax[1].bar(x + 0.15, l2_02, width=0.25, label='Cluster 2', color='darkgreen')

# ax[1].set_xticks(x, labels=['Car driver', 'Car passenger', 'Train', 
#                       'BTM', 'Cycling', 'Walking'], rotation=45,
#                       ha='right')

# # ax[1].set_title('Modal split OViN, corrected by demography')
# ax[1].set_ylabel('Percentage of mode use')
# ax[1].legend()
# ax[1].set_yticks(np.arange(0, 50, 5))
# ax[1].grid(axis='y')
# ax[1].set_axisbelow(True)

# ax[2].bar(x - 0.15, l2_12, width=0.25, label='Cluster 2', color='darkgreen')
# # ax[0].bar(x, r2, width=0.2, label='Cluster 2')
# ax[2].bar(x + 0.15, l1_12, width=0.25, label='Cluster 1', color='royalblue')

# ax[2].set_xticks(x, labels=['Car driver', 'Car passenger', 'Train', 
#                       'BTM', 'Cycling', 'Walking'], rotation=45,
#                       ha='right')

# # ax[2].set_title('Modal split OViN, corrected by demography')
# ax[2].set_ylabel('Percentage of mode use')
# ax[2].legend()
# ax[2].set_yticks(np.arange(0, 50, 5))
# ax[2].grid(axis='y')
# ax[2].set_axisbelow(True)


# Real data
f, ax = plt.subplots(1, 2)
f.set_figwidth(12)

x = np.arange(6)

ax[0].bar(x - 0.25, r0, width=0.2, label='Cluster 0', color='firebrick')
ax[0].bar(x, r2, width=0.2, label='Cluster 2', color='darkgreen')
ax[0].bar(x + 0.25, r1, width=0.2, label='Cluster 1', color='royalblue')

ax[0].set_xticks(x, labels=['Car driver', 'Car passenger', 'Train', 
                      'BTM', 'Cycling', 'Walking'], rotation=45,
                      ha='right')

ax[0].set_title('Modal split OViN 3 clusters, uncorrected')
ax[0].set_ylabel('Percentage of mode use')
ax[0].legend()
ax[0].set_yticks(np.arange(0, 50, 5))
ax[0].grid(axis='y')
ax[0].set_axisbelow(True)

ax[1].bar(x - 0.25, l0, width=0.2, label='Cluster 0', color='firebrick')
ax[1].bar(x, l2, width=0.2, label='Cluster 2', color='darkgreen')
ax[1].bar(x + 0.25, l1, width=0.2, label='Cluster 1', color='royalblue')

ax[1].set_xticks(x, labels=['Car driver', 'Car passenger', 'Train', 
                      'BTM', 'Cycling', 'Walking'], rotation=45,
                      ha='right')

ax[1].set_title('Modal split LMS 3 clusters, uncorrected')
ax[1].set_ylabel('Percentage of mode use')
ax[1].legend()
ax[1].set_yticks(np.arange(0, 50, 5))
ax[1].grid(axis='y')
ax[1].set_axisbelow(True);

Look at effect sizes

In [ ]:
def cohen_d(d1, d2):

    n1, n2 = len(d1), len(d2) # Calculate size of samples

    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1) # Calculate variances

    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))

    u1, u2 = np.mean(d1), np.mean(d2)

    return (u1 - u2) / s

In [ ]:
def SMD(d1, d2):

    s1, s2 = np.std(d1), np.std(d2)

    return 100 * (np.mean(d1) - np.mean(d2)) / np.sqrt((s1 ** 2 + s2 ** 2) / 2)

In [ ]:
cols = ['HHPers', 'Leeftijd', 'HHBestInk', 'HHAuto']
# cols = ['Household_Size', 'Age_average', 'Income_hh_average', 'Cars_HH']

In [ ]:
effect_sizes_b = []
effect_sizes_a = []



for cl in cols:
    _, p_before = sst.ttest_ind(o0[cl], o1[cl])
    _, p_after = sst.ttest_ind(df01_c[cl], df01_m[cl])

    cohen_d_before = cohen_d(o1[cl], o0[cl])
    cohen_d_after = cohen_d(df01_m[cl], df01_c[cl])

    smd_before = SMD(o1[cl], o0[cl])
    smd_after = SMD(df01_m[cl], df01_c[cl])

    effect_sizes_b.append([cl, 'before', cohen_d_before, p_before, smd_before])
    effect_sizes_a.append([cl, 'after', cohen_d_after, p_after, smd_after])


In [ ]:
df_effect_b01 = pd.DataFrame(effect_sizes_b)
df_effect_a01 = pd.DataFrame(effect_sizes_a)

In [ ]:
df_effect_b01

In [ ]:
df_effect_a01

In [ ]:
x = np.arange(4)
plt.bar(x - 0.15, df_effect_b01[4], width=0.25, label='Before matching')
plt.bar(x + 0.15, df_effect_a01[4], width=0.25, label='After matching')
plt.axhline(0, color='black')
plt.axhline(10, color='green', linestyle='--', label='SMD = |10|%')
plt.axhline(-10, color='green', linestyle='--')

plt.title('Standard mean differences')
plt.legend()
plt.xticks(x, labels=['Household size', 'Age', 'Income', 'Number of cars'],
           rotation=45, ha='right');

In [ ]:
effect_sizes_b = []
effect_sizes_a = []

# cols = ['HHPers', 'Leeftijd', 'HHBestInk', 'HHAuto']

for cl in cols:
    _, p_before = sst.ttest_ind(o0[cl], o2[cl])
    _, p_after = sst.ttest_ind(df02_c[cl], df02_m[cl])

    cohen_d_before = cohen_d(o2[cl], o0[cl])
    cohen_d_after = cohen_d(df02_m[cl], df02_c[cl])

    smd_before = SMD(o2[cl], o0[cl])
    smd_after = SMD(df02_m[cl], df02_c[cl])

    effect_sizes_b.append([cl, 'before', cohen_d_before, p_before, smd_before])
    effect_sizes_a.append([cl, 'after', cohen_d_after, p_after, smd_after])


In [ ]:
df_effect_b02 = pd.DataFrame(effect_sizes_b)
df_effect_a02 = pd.DataFrame(effect_sizes_a)

df_effect_b02

In [ ]:
df_effect_a02

In [ ]:
x = np.arange(4)
plt.bar(x - 0.15, df_effect_b02[4], width=0.25, label='Before')
plt.bar(x + 0.15, df_effect_a02[4], width=0.25, label='After')
plt.axhline(0, color='black')
plt.axhline(10, color='green', linestyle='--', label='SMD = |10|%')
plt.axhline(-10, color='green', linestyle='--')
plt.title('Cluster 0 and 2')
plt.legend()
plt.xticks(x, labels=['Household size', 'Age', 'Income', 'Number of cars'],
           rotation=45, ha='right');

In [ ]:
effect_sizes_b = []
effect_sizes_a = []

# cols = ['HHPers', 'Leeftijd', 'HHBestInk', 'HHAuto']

for cl in cols:
    _, p_before = sst.ttest_ind(o1[cl], o2[cl])
    _, p_after = sst.ttest_ind(df12_c[cl], df12_m[cl])

    cohen_d_before = cohen_d(o2[cl], o1[cl])
    cohen_d_after = cohen_d(df12_m[cl], df12_c[cl])

    smd_before = SMD(o2[cl], o1[cl])
    smd_after = SMD(df12_m[cl], df12_c[cl])

    effect_sizes_b.append([cl, 'before', cohen_d_before, p_before, smd_before])
    effect_sizes_a.append([cl, 'after', cohen_d_after, p_after, smd_after])

In [ ]:
df_effect_b12 = pd.DataFrame(effect_sizes_b)
df_effect_a12 = pd.DataFrame(effect_sizes_a)

df_effect_b12

In [ ]:
df_effect_a12

In [ ]:
x = np.arange(4)
plt.bar(x - 0.15, df_effect_b12[4], width=0.25, label='Before matching')
plt.bar(x + 0.15, df_effect_a12[4], width=0.25, label='After matching')
plt.axhline(0, color='black')
plt.axhline(10, color='green', linestyle='--', label='SMD = |10|%')
plt.axhline(-10, color='green', linestyle='--')
plt.title('Cluster 1 and 2')
plt.legend()
plt.xticks(x, labels=['Household size', 'Age', 'Income', 'Number of cars'],
           rotation=45, ha='right');

In [ ]:
f, ax = plt.subplots(1, 3)
f.set_figwidth(14)
f.set_figheight(4)

ax[0].bar(x - 0.15, df_effect_b01[4], width=0.25, label='Before matching')
ax[0].bar(x + 0.15, df_effect_a01[4], width=0.25, label='After matching')
ax[0].axhline(0, color='black')
ax[0].axhline(10, color='green', linestyle='--', label='SMD = |10|%')
ax[0].axhline(-10, color='green', linestyle='--')
ax[0].set_title('Cluster 0 and 1')
ax[0].legend()
ax[0].set_xticks(x, labels=['Household size', 'Age', 'Income', 'Number of cars'],
           rotation=45, ha='right');


ax[1].bar(x - 0.15, df_effect_b02[4], width=0.25, label='Before matching')
ax[1].bar(x + 0.15, df_effect_a02[4], width=0.25, label='After matching')
ax[1].axhline(0, color='black')
ax[1].axhline(10, color='green', linestyle='--', label='SMD = |10|%')
ax[1].axhline(-10, color='green', linestyle='--')
ax[1].set_title('Cluster 0 and 2')
ax[1].legend()
ax[1].set_xticks(x, labels=['Household size', 'Age', 'Income', 'Number of cars'],
           rotation=45, ha='right');


ax[2].bar(x - 0.15, df_effect_b12[4], width=0.25, label='Before matching')
ax[2].bar(x + 0.15, df_effect_a12[4], width=0.25, label='After matching')
ax[2].axhline(0, color='black')
ax[2].axhline(10, color='green', linestyle='--', label='SMD = |10|%')
ax[2].axhline(-10, color='green', linestyle='--')
ax[2].set_title('Cluster 1 and 2')
ax[2].legend()
ax[2].set_xticks(x, labels=['Household size', 'Age', 'Income', 'Number of cars'],
           rotation=45, ha='right');


for i in range(3):
    ax[i].set_yticks(np.arange(-350, 200, 50))

Calculate ATE

In [ ]:
ate01 = counts1_01 - counts0_01
ate02 = counts2_02 - counts0_02
ate12 = counts2_12 - counts1_12

In [ ]:
obe01 = r1 - r0
obe02 = r2 - r0
obe12 = r2 - r1

In [ ]:
ate01

In [ ]:
ratio01 = np.array(ate01) / np.array(obe01)
ratio02 = np.array(ate02) / np.array(obe02)
ratio12 = np.array(ate12) / np.array(obe12)


In [ ]:
df_results = pd.DataFrame({
    'mode': ['Car driver', 'Car passenger', 'Train', 'BTM', 'Bike', 'Walking'],
    'OBE01':np.array(obe01),
    'ATE01': np.array(ate01),
    'Ratio 01':ratio01 * 100,
    'OBE02':np.array(obe02),
    'ATE02': np.array(ate02),
    'Ratio 02':ratio02 * 100,
    'OBE12':np.array(obe12),
    'ATE12': np.array(ate12),
    'Ratio 12':ratio12 * 100,
})

In [ ]:
df_results = np.round(df_results, 2)
df_results

In [ ]:
df02.iloc[:, 6]

Check with Multivariate regression

In [ ]:
# X01 = df01.iloc[:, np.r_[1:5, 6]]
X01 = df01.iloc[:, np.r_[1:6]]
# y01 = df01.iloc[:, 5]
y01 = df01.iloc[:, 6]


# X02 = df02.iloc[:, np.r_[1:5, 6]]
X02 = df02.iloc[:, np.r_[1:6]]
# y02 = df02.iloc[:, 5]
y02 = df02.iloc[:, 6]

# X12 = df12.iloc[:, np.r_[1:5, 6]]
X12 = df12.iloc[:, np.r_[1:6]]
# y12 = df12.iloc[:, 5]
y12 = df12.iloc[:, 6]

In [ ]:
regr01 = LinearRegression()
regr01.fit(X01, y01)

regr02 = LinearRegression()
regr02.fit(X02, y02)

regr12 = LinearRegression()
regr12.fit(X12, y12)

In [ ]:
regr01.coef_

In [ ]:
regr02.coef_

In [ ]:
regr12.coef_